# DemoWhereIsMySqlAgent.ipynb

<span style="font-size: 14px;">Written by Taiob&nbsp; Ali</span>

<span style="font-size: 14px;">SqlWorldWide.com</span>

## Set up Demo

Setup section of the notebook will create:

-  A resource group
- Two logical SQL server
- Few sample databases
- SQL Elastic Pool
- Clean up code at the end

<mark>Script can take between 40~50 minutes during my test. Mileage will vary in your case</mark>

Credit:

- [https://docs.microsoft.com/en-us/azure/sql-database/sql-database-get-started-powershell](https://docs.microsoft.com/en-us/azure/sql-database/sql-database-get-started-powershell)
- [https://gallery.technet.microsoft.com/scriptcenter/Get-ExternalPublic-IP-c1b601bb](https://gallery.technet.microsoft.com/scriptcenter/Get-ExternalPublic-IP-c1b601bb)

<span style="color: rgb(23, 23, 23); font-family: &quot;Segoe UI&quot;, SegoeUI, &quot;Helvetica Neue&quot;, Helvetica, Arial, sans-serif; font-size: 16px; background-color: rgb(226, 218, 241);">PowerShell 7 and later is the recommended version of PowerShell for use with Az PowerShell on all platforms.</span>

To check the version of PowerShell running on your machine, run the following command.

If you have an outdated version, upgrade from:  
[https://github.com/PowerShell/PowerShell](https://github.com/PowerShell/PowerShell)

In [ ]:
$PSVersionTable.PSVersion


Major  Minor  Build  Revision
-----  -----  -----  --------
5      1      19041  610     




Install Az Module if you do not have it.  
<span style="font-size: 14px;">https://docs.microsoft.com/en-us/powershell/azure/install-az-ps?view=azps-5.4.0</span>

<span style="font-size: 14px;">If you've disabled module autoloading, manually import the module with `Import-Module -Name Az`. Because of the way the module is structured, this can take a few seconds.</span>

In [ ]:
Import-Module -name Az 

<span style="font-size: 14px;">Setup your Debug Preference</span>

[https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about\_preference\_variables?view=powershell-7.1#debugpreference](https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_preference_variables?view=powershell-7.1#debugpreference)  
<span style="font-size: 14px;">This example shows the effect of `$DebugPreference` with the Continue value. The debug message is displayed and the command continues to process.</span>

In [ ]:
$DebugPreference = "Continue"
Write-Debug -Message "Hello, World"

<span style="font-size: 14px;">Sign in to Azure</span>

In [1]:
Connect-AzAccount

. {


>> Connect-AzAccount
>> }
>> 


selected for further use. To select another subscription, use Set-AzContext.


Account                  SubscriptionName         TenantId                             Environment
-------                  ----------------         --------                             -----------
taiobmdjamshed@yahoo.com Visual Studio Enterprise d5b50601-6698-4f8f-beb6-1799fee4dc80 AzureCloud 




If you need to seethe list of your subscription

<span style="color: rgb(106, 153, 85);">$SubscriptionList=Get-AzSubscription</span><span style="color: rgb(106, 153, 85);">$SubscriptionList</span>

Use below code if you have multiple subscription and you want to use a particular one

In [2]:
Set-AzContext -SubscriptionId '18d92f52-ac34-4379-ab8b-5a5106f1c54e'


Name                                     Account             SubscriptionName    Environment         TenantId          
----                                     -------             ----------------    -----------         --------          
Visual Studio Enterprise (18d92f52-ac... taiobmdjamshed@y... Visual Studio En... AzureCloud          d5b50601-6698-4...


Declare Variables

In [3]:
$resourceGroupName = "sqlagentdemo"
$primaryLocation = "East US"  
$elasticPoolName = "sqlagentdemo"
$jobServerName = "ugdemojobserver"
$targetServerName = "ugdemotargetserver"
$adminlogin = "taiob"
$password = Get-Content "C:\password.txt"
$ipinfo = Invoke-RestMethod http://ipinfo.io/json 
$startip = $ipinfo.ip
$endip = $ipinfo.ip 
$jobDatabase = "jobdatabase"
$collectionDatabase = "dbawarehouse"
$databaseName1 = "adventureworks"
$databaseName2 = "WideWorldImporters"
$localInstanceName = 'DESKTOP-50O69FS\SQL2019'
$localDatabaseName = 'dbadatabase'
$localTableName = 'databasesize'
$automationAccountName = "ugdemo2"
$credentialName = "sqlservercredentials"

Check if Resource group exists. If exist delete the resource group.

Create a new Resource group

In [4]:
$resGrpChk = Get-AzResourceGroup `
    -Name $resourceGroupName `
    -ev notPresent `
    -ea 0

if ($resGrpChk) {  
    #Delete resource group
    Remove-AzResourceGroup `
        -Name $resourceGroupName -Confirm   
    Write-Host 'Resource group deleted' `
        -fore white `
        -back green
}

New-AzResourceGroup `
    -Name $resourceGroupName `
    -Location "East US 2"    
Write-Host 'Resource group created' `
    -fore white `
    -back green



ResourceGroupName : sqlagentdemo
Location          : eastus2
ProvisioningState : Succeeded
Tags              : 
ResourceId        : /subscriptions/18d92f52-ac34-4379-ab8b-5a5106f1c54e/resourceGroups/sqlagentdemo



Resource group created




- <span style="font-size: 14px;">Creating job server</span>
- <span style="font-size: 14px;">Configure server firewall rule for job server</span>
- <span style="font-size: 14px;">Create an empty database to hold job metadata</span>
- <span style="font-size: 14px;">Create an empty database to hold job output</span>

In [5]:
New-AzSqlServer `
    -ResourceGroupName $resourceGroupName `
    -ServerName $jobServerName `
    -Location $primaryLocation `
    -SqlAdministratorCredentials $(New-Object -TypeName System.Management.Automation.PSCredential `
    -ArgumentList $adminlogin, $(ConvertTo-SecureString -String $password -AsPlainText -Force))

#Configure a server firewall rule for job server
New-AzSqlServerFirewallRule `
    -ResourceGroupName $resourceGroupName `
    -ServerName $jobServerName `
    -FirewallRuleName "TaiobDesktop" `
    -StartIpAddress $startip `
    -EndIpAddress $endip

# This is done to allow access to Azure Services
New-AzSqlServerFirewallRule `
    -ServerName $jobServerName `
    -ResourceGroupName $resourceGroupName  `
    -AllowAllAzureIPs

New-AzSqlDatabase  -ResourceGroupName $resourceGroupName `
    -ServerName $jobServerName `
    -DatabaseName $jobDatabase `
    -Edition "Standard" `
    -RequestedServiceObjectiveName "S0" `
    -MaxSizeBytes 10737418240 

New-AzSqlDatabase  -ResourceGroupName $resourceGroupName `
    -ServerName $jobServerName `
    -DatabaseName $collectionDatabase `
    -Edition "Standard" `
    -RequestedServiceObjectiveName "S0" `
    -MaxSizeBytes 10737418240 



ResourceGroupName        : sqlagentdemo
ServerName               : ugdemojobserver
Location                 : eastus
SqlAdministratorLogin    : taiob
SqlAdministratorPassword : 
ServerVersion            : 12.0
Tags                     : 
Identity                 : 
FullyQualifiedDomainName : ugdemojobserver.database.windows.net
ResourceId               : /subscriptions/18d92f52-ac34-4379-ab8b-5a5106f1c54e/resourceGroups/sqlagentdemo/providers/Mi
                           crosoft.Sql/servers/ugdemojobserver
MinimalTlsVersion        : 
PublicNetworkAccess      : Enabled



ResourceGroupName : sqlagentdemo
ServerName        : ugdemojobserver
StartIpAddress    : 71.174.126.37
EndIpAddress      : 71.174.126.37
FirewallRuleName  : TaiobDesktop



ResourceGroupName : sqlagentdemo
ServerName        : ugdemojobserver
StartIpAddress    : 0.0.0.0
EndIpAddress      : 0.0.0.0
FirewallRuleName  : AllowAllAzureIPs



ResourceGroupName             : sqlagentdemo
ServerName                    : ugdemojobserver
DatabaseName                  : jobdatabase
Location                      : eastus
DatabaseId                    : dc569730-4186-4685-ad9f-95d2079e24a1
Edition                       : Standard
CollationName                 : SQL_Latin1_General_CP1_CI_AS
CatalogCollation              : 
MaxSizeBytes                  : 10737418240
Status                        : Online
CreationDate                  : 3/8/2021 11:15:38 PM
CurrentServiceObjectiveId     : 00000000-0000-0000-0000-000000000000
CurrentServiceObjectiveName   : S0
RequestedServiceObjectiveName : S0
RequestedServiceObjectiveId   : 
ElasticPoolName               : 
EarliestRestoreDate           : 3/8/2021 11:45:38 PM
Tags                          : 
ResourceId                    : /subscriptions/18d92f52-ac34-4379-ab8b-5a5106f1c54e/resourceGroups/sqlagentdemo/provide
                                rs/Microsoft.Sql/servers/ugdemojobserver/

ResourceGroupName             : sqlagentdemo
ServerName                    : ugdemojobserver
DatabaseName                  : dbawarehouse
Location                      : eastus
DatabaseId                    : 8c19faaa-2868-4f3e-a3a9-e13d7dcf915d
Edition                       : Standard
CollationName                 : SQL_Latin1_General_CP1_CI_AS
CatalogCollation              : 
MaxSizeBytes                  : 10737418240
Status                        : Online
CreationDate                  : 3/8/2021 11:16:37 PM
CurrentServiceObjectiveId     : 00000000-0000-0000-0000-000000000000
CurrentServiceObjectiveName   : S0
RequestedServiceObjectiveName : S0
RequestedServiceObjectiveId   : 
ElasticPoolName               : 
EarliestRestoreDate           : 3/8/2021 11:46:37 PM
Tags                          : 
ResourceId                    : /subscriptions/18d92f52-ac34-4379-ab8b-5a5106f1c54e/resourceGroups/sqlagentdemo/provide
                                rs/Microsoft.Sql/servers/ugdemojobserver

- <span style="font-size: 14px;">Create target server</span>
- <span style="font-size: 14px;">Configure server firewall rule for target server<br></span>
- <span style="font-size: 14px;">Create a database using adventureworks smaple</span>
- <span style="font-size: 14px;">Create a database for runbook demo&nbsp;</span>

In [6]:
New-AzSqlServer `
    -ResourceGroupName $resourceGroupName `
    -ServerName $targetServerName `
    -Location $primaryLocation `
    -SqlAdministratorCredentials $(New-Object -TypeName System.Management.Automation.PSCredential `
    -ArgumentList $adminlogin, $(ConvertTo-SecureString -String $password -AsPlainText -Force))

#Configure a server firewall rule for target server
New-AzSqlServerFirewallRule `
    -ResourceGroupName $resourceGroupName `
    -ServerName $targetServerName `
    -FirewallRuleName "TaiobDesktop" `
    -StartIpAddress $startip `
    -EndIpAddress $endip

#This is done to allow access to Azure Services 
New-AzSqlServerFirewallRule `
    -ServerName $targetServerName `
    -ResourceGroupName $resourceGroupName  `
    -AllowAllAzureIPs

New-AzSqlDatabase  `
    -ResourceGroupName $resourceGroupName `
    -ServerName $targetServerName `
    -DatabaseName $databasename1 `
    -Edition "Standard" `
    -RequestedServiceObjectiveName "S0" `
    -MaxSizeBytes 10737418240 `
    -SampleName "AdventureWorksLT"

New-AzSqlDatabase  `
    -ResourceGroupName $resourceGroupName `
    -ServerName $targetServerName `
    -DatabaseName testRunBookDB `
    -Edition "Standard" `
    -RequestedServiceObjectiveName "S0" `
    -MaxSizeBytes 10737418240 `

. {
>> New-AzSqlServer `
>>     -ResourceGroupName $resourceGroupName `
>>     -ServerName $targetServerName `
>>     -Location $primaryLocation `
>>     -SqlAdministratorCredentials $(New-Object -TypeName System.Management.Automation.PSCredential `
>>     -ArgumentList $adminlogin, $(ConvertTo-SecureString -String $password -AsPlainText -Force))
>> 
>> #Configure a server firewall rule for target server
>> New-AzSqlServerFirewallRule `
>>     -ResourceGroupName $resourceGroupName `
>>     -ServerName $targetServerName `
>>     -FirewallRuleName "TaiobDesktop" `
>>     -StartIpAddress $startip `
>>     -EndIpAddress $endip
>> 
>> #This is done to allow access to Azure Services 
>> New-AzSqlServerFirewallRule `
>>     -ServerName $targetServerName `
>>     -ResourceGroupName $resourceGroupName  `
>>     -AllowAllAzureIPs
>> 
>> New-AzSqlDatabase  `
>>     -ResourceGroupName $resourceGroupName `
>>     -ServerName

 $targetServerName `
>>     -DatabaseName $databasename1 `
>>     -Edition "Standard" `
>>     -RequestedServiceObjectiveName "S0" `
>>     -MaxSizeBytes 10737418240 `
>>     -SampleName "AdventureWorksLT"
>> 
>> New-AzSqlDatabase  `
>>     -ResourceGroupName $resourceGroupName `
>>     -ServerName $targetServerName `
>>     -DatabaseName testRunBookDB `
>>     -Edition "Standard" `
>>     -RequestedServiceObjectiveName "S0" `
>>     -MaxSizeBytes 10737418240 `
>> }
>> 




ResourceGroupName        : sqlagentdemo
ServerName               : ugdemotargetserver
Location                 : eastus
SqlAdministratorLogin    : taiob
SqlAdministratorPassword : 
ServerVersion            : 12.0
Tags                     : 
Identity                 : 
FullyQualifiedDomainName : ugdemotargetserver.database.windows.net
ResourceId               : /subscriptions/18d92f52-ac34-4379-ab8b-5a5106f1c54e/resourceGroups/sqlagentdemo/providers/Mi
                           crosoft.Sql/servers/ugdemotargetserver
MinimalTlsVersion        : 
PublicNetworkAccess      : Enabled



ResourceGroupName : sqlagentdemo
ServerName        : ugdemotargetserver
StartIpAddress    : 71.174.126.37
EndIpAddress      : 71.174.126.37
FirewallRuleName  : TaiobDesktop



ResourceGroupName : sqlagentdemo
ServerName        : ugdemotargetserver
StartIpAddress    : 0.0.0.0
EndIpAddress      : 0.0.0.0
FirewallRuleName  : AllowAllAzureIPs



ResourceGroupName             : sqlagentdemo
ServerName                    : ugdemotargetserver
DatabaseName                  : adventureworks
Location                      : eastus
DatabaseId                    : 17723696-129c-44bf-8080-861c14ef720e
Edition                       : Standard
CollationName                 : SQL_Latin1_General_CP1_CI_AS
CatalogCollation              : 
MaxSizeBytes                  : 10737418240
Status                        : Online
CreationDate                  : 3/8/2021 11:19:04 PM
CurrentServiceObjectiveId     : 00000000-0000-0000-0000-000000000000
CurrentServiceObjectiveName   : S0
RequestedServiceObjectiveName : S0
RequestedServiceObjectiveId   : 
ElasticPoolName               : 
EarliestRestoreDate           : 3/8/2021 11:49:04 PM
Tags                          : 
ResourceId                    : /subscriptions/18d92f52-ac34-4379-ab8b-5a5106f1c54e/resourceGroups/sqlagentdemo/provide
                                rs/Microsoft.Sql/servers/ugdemotarg

ResourceGroupName             : sqlagentdemo
ServerName                    : ugdemotargetserver
DatabaseName                  : testRunBookDB
Location                      : eastus
DatabaseId                    : 00369034-7dc4-4ebd-9f07-537b9d261c15
Edition                       : Standard
CollationName                 : SQL_Latin1_General_CP1_CI_AS
CatalogCollation              : 
MaxSizeBytes                  : 10737418240
Status                        : Online
CreationDate                  : 3/8/2021 11:20:05 PM
CurrentServiceObjectiveId     : 00000000-0000-0000-0000-000000000000
CurrentServiceObjectiveName   : S0
RequestedServiceObjectiveName : S0
RequestedServiceObjectiveId   : 
ElasticPoolName               : 
EarliestRestoreDate           : 3/8/2021 11:50:05 PM
Tags                          : 
ResourceId                    : /subscriptions/18d92f52-ac34-4379-ab8b-5a5106f1c54e/resourceGroups/sqlagentdemo/provide
                                rs/Microsoft.Sql/servers/ugdemotarge

<span style="font-size: 14px;">Create a database using wideworldimporters bacpac file</span>

<span style="font-size: 14px;">https://github.com/Microsoft/sql-server-samples/releases/download/wide-world-importers-v1.0/WideWorldImporters-Standard.bacpac</span>

In [7]:
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
$url = "http://github.com/Microsoft/sql-server-samples/releases/download/wide-world-importers-v1.0/WideWorldImporters-Standard.bacpac"
$output = "c:\WideWorldImporters-Standard.bacpac"
Invoke-WebRequest -Uri $url -OutFile $output
Set-Location "C:\Program Files (x86)\Microsoft SQL Server\140\DAC\bin\"

.\sqlpackage.exe /a:Import /sf:$output /tsn:"$targetServerName.database.windows.net" `
    /tdn:$databaseName2 /tu:$adminlogin /tp:$password
    

. {
>> [Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
>> $url = "http://github.com/Microsoft/sql-server-samples/releases/download/wide-world-importers-v1.0/WideWorldImporters-Standard.bacpac"
>> $output = "c:\WideWorldImporters-Standard.bacpac"
>> Invoke-WebRequest -Uri $url -OutFile $output
>> Set-Location "C:\Program Files (x86)\Microsoft SQL Server\140\DAC\bin\"
>> 


>> .\sqlpackage.exe /a:Import /sf:$output /tsn:"$targetServerName.database.windows.net" `
>>     /tdn:$databaseName2 /tu:$adminlogin /tp:$password
>>     
>> }
>> 


Importing to database 'WideWorldImporters' on server 'ugdemotargetserver.database.windows.net'.


Creating deployment plan
Initializing deployment


*** A project which specifies SQL Server 2016 as the target platform may experience compatibility issues with Microsoft Azure SQL Database v12.


...

Enabling index 'FK_Warehouse_StockItemTransactions_PurchaseOrderID'.


Enabling index 'FK_Warehouse_StockItemTransactions_StockItemID'.


Enabling index 'FK_Warehouse_StockItemTransactions_SupplierID'.


Enabling index 'FK_Warehouse_StockItemTransactions_TransactionTypeID'.


Successfully imported database.


- Create an elastic database pool for a SQL Database
- Create an empty database name test1

In [8]:
New-AzSqlElasticPool `
    -ResourceGroupName $resourceGroupName `
    -ServerName $targetServerName `
    -ElasticPoolName $elasticPoolName `
    -Edition "Standard" `
    -Dtu 400 `
    -DatabaseDtuMin 10 `
    -DatabaseDtuMax 100

New-AzSqlDatabase  `
    -ResourceGroupName $resourceGroupName `
    -ServerName $targetServerName `
    -DatabaseName "test1" `
    -MaxSizeBytes 10737418240 `
    -ElasticPoolName $elasticPoolName



ResourceId          : /subscriptions/18d92f52-ac34-4379-ab8b-5a5106f1c54e/resourceGroups/sqlagentdemo/providers/Microso
                      ft.Sql/servers/ugdemotargetserver/elasticPools/sqlagentdemo
ResourceGroupName   : sqlagentdemo
ServerName          : ugdemotargetserver
ElasticPoolName     : sqlagentdemo
Location            : eastus
CreationDate        : 3/8/2021 11:34:30 PM
State               : Ready
Edition             : Standard
SkuName             : StandardPool
Dtu                 : 400
DatabaseDtuMax      : 100
DatabaseDtuMin      : 10
Capacity            : 400
DatabaseCapacityMin : 10
DatabaseCapacityMax : 100
Family              : 
MaxSizeBytes        : 429496729600
StorageMB           : 409600
Tags                : 
ZoneRedundant       : False
LicenseType         : 



ResourceGroupName             : sqlagentdemo
ServerName                    : ugdemotargetserver
DatabaseName                  : test1
Location                      : eastus
DatabaseId                    : 3ae1dd41-8e21-4749-9e90-6e04734a4730
Edition                       : Standard
CollationName                 : SQL_Latin1_General_CP1_CI_AS
CatalogCollation              : 
MaxSizeBytes                  : 10737418240
Status                        : Online
CreationDate                  : 3/8/2021 11:35:38 PM
CurrentServiceObjectiveId     : 00000000-0000-0000-0000-000000000000
CurrentServiceObjectiveName   : ElasticPool
RequestedServiceObjectiveName : ElasticPool
RequestedServiceObjectiveId   : 
ElasticPoolName               : sqlagentdemo
EarliestRestoreDate           : 3/9/2021 12:05:38 AM
Tags                          : 
ResourceId                    : /subscriptions/18d92f52-ac34-4379-ab8b-5a5106f1c54e/resourceGroups/sqlagentdemo/provide
                                rs/Microsoft.S

To use Elastic Jobs, register the feature in your Azure subscription by running the following command (this only needs to be run once in each subscription where you want to use Elastic Jobs)

In [ ]:
Register-AzProviderFeature `
    -FeatureName sqldb-JobAccounts `
    -ProviderNamespace Microsoft.Sql

## Demo starts here

### Demo running script against Azure SQL Database using Linked Server

Use NoteBook  1\_LinkedServer.ipynb

<h3>Demo how to run scripts against all Azure SQL Database in your tenant</h3>

<span style="font-size: 14px;">This script will&nbsp;&nbsp;</span> 

-  Iterate through all resource type SQL Server
- Collect file size of from all Azure SQL Database from all Azure SQL Servers
- Save the result in local On-Premises server

In [11]:

# Putting my query in a variable
$databaseQuery = 
"
SELECT 
		GETDATE() AS collectedAT,
		@@SERVERNAME AS serverName, 
		DB_NAME() AS databaseName, 
		LEFT(a.name, 64) AS fileName,
		a.file_id AS fileId,
		a.size AS fileSizeMB,
		CONVERT(DECIMAL(12, 2), ROUND(FILEPROPERTY(a.name,'SpaceUsed')/ 128.000, 2)) AS spaceUsedMB,
		CONVERT(DECIMAL(12, 2), ROUND(( a.size - FILEPROPERTY(a.name,'SpaceUsed'))/ 128.000, 2)) AS freeSpaceMB,
		CONVERT(DECIMAL(12, 2), (CONVERT(DECIMAL(12, 2), ROUND((a.size - FILEPROPERTY(a.name,'SpaceUsed'))/128.000, 2))*100)/ CONVERT(DECIMAL(12, 2), ROUND(a.size / 128.000, 2))) as percentFree,
		a.physical_name AS physicalName 
FROM sys.database_files a
"

$password = ConvertTo-SecureString -String $password -AsPlainText -Force
#$databaseCredentials = Get-Credential -Message "Please provide credentials for $SqlInstance"
$databaseCredentials = New-Object System.Management.Automation.PSCredential($adminlogin, $password) 

#Get all resources type SQL Server, loop through all SQL Server and collect size for each database
$resources = Get-AzResource -ResourceGroupName 'sqlagentdemo' | Where-Object { $_.ResourceType -eq "Microsoft.Sql/servers" } | Select-Object name
foreach ($SqlInstance in $resources) { 
    $SqlInstance = "$($SqlInstance.Name).database.windows.net"
    $databases = Invoke-Sqlcmd -Query "select name from sys.databases" -ServerInstance $SqlInstance `
        -Username $databaseCredentials.GetNetworkCredential().UserName `
        -Password $databaseCredentials.GetNetworkCredential().Password `
        -Database 'master'

    foreach ($databaseName in $databases.name) {
        Write-Host "Query results for database $databaseName.`n"
        Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
            -Username $databaseCredentials.GetNetworkCredential().UserName `
            -Password $databaseCredentials.GetNetworkCredential().Password `
            -Database $databaseName | `
            Write-DbaDbTableData -SqlInstance $localInstanceName -Database $localDatabaseName -Table $localTableName
    }
}

. {
>> 
>> # Putting my query in a variable
>> $databaseQuery = 
>> "
>> SELECT 
>> 		GETDATE() AS collectedAT,
>> 		@@SERVERNAME AS serverName, 
>> 		DB_NAME() AS databaseName, 
>> 		LEFT(a.name, 64) AS fileName,
>> 		a.file_id AS fileId,
>> 		a.size AS fileSizeMB,
>> 		CONVERT(DECIMAL(12, 2), ROUND(FILEPROPERTY(a.name,'SpaceUsed')/ 128.000, 2)) AS spaceUsedMB,
>> 		CONVERT(DECIMAL(12, 2), ROUND(( a.size - FILEPROPERTY(a.name,'SpaceUsed'))/ 128.000, 2)) AS freeSpaceMB,
>> 		CONVERT(DECIMAL(12, 2), (CONVERT(DECIMAL(12, 2), ROUND((a.size - FILEPROPERTY(a.name,'SpaceUsed'))/128.000, 2))*100)/ CONVERT(DECIMAL(12, 2), ROUND(a.size / 128.000, 2))) as percentFree,
>> 		a.physical_name AS physicalName 
>> FROM sys.database_files a
>> "
>> 
>> $password = ConvertTo-SecureString -String $password -AsPlainText -Force
>> #$databaseCredentials = Get-Credential -Message "Please provide credentials for $Sq

lInstance"
>> $databaseCredentials = New-Object System.Management.Automation.PSCredential($adminlogin, $password) 
>> 
>> #Get all resources type SQL Server, loop through all SQL Server and collect size for each database
>> $resources = Get-AzResource -ResourceGroupName 'sqlagentdemo' | Where-Object { $_.ResourceType -eq "Microsoft.Sql/servers" } | Select-Object name
>> foreach ($SqlInstance in $resources) { 
>>     $SqlInstance = "$($SqlInstance.Name).database.windows.net"
>>     $databases = Invoke-Sqlcmd -Query "select name from sys.databases" -ServerInstance $SqlInstance `
>>         -Username $databaseCredentials.GetNetworkCredential().UserName `
>>         -Password $databaseCredentials.GetNetworkCredential().Password `
>>         -Database 'master'
>> 
>>     foreach ($databaseName in $databases.name) {
>>         Write-Host "Query results for database $databaseName.`n"
>>         Invoke-Sqlcmd $databaseQuery -ServerInstance $SqlInstance `
>>             -Username $databaseCrede

Query results for database master.



Query results for database jobdatabase.



Query results for database dbawarehouse.



Query results for database master.



Query results for database adventureworks.



Query results for database testRunBookDB.



Query results for database WideWorldImporters.



Query results for database test1.



See the result

In [14]:
Invoke-DbaQuery `
    -SqlInstance $localInstanceName `
    -Query 'SELECT TOP 20 * FROM DbaDatabase.dbo.databasesize ORDER BY collectedAT DESC ;' | Format-Table -AutoSize

. {


>> Invoke-DbaQuery `
>>     -SqlInstance $localInstanceName `
>>     -Query 'SELECT TOP 20 * FROM DbaDatabase.dbo.databasesize ORDER BY collectedAT DESC ;' | Format-Table -AutoSize
>> }
>> 



collectedAT          serverName         databaseName       fileName fileId fileSizeMB spaceUsedMB freeSpaceMB percentFr
                                                                         

                                            ee
-----------          ----------         ------------       -------- ------ ---------- ----------- ----------- ---------
2/27/2021 5:28:41 PM ugdemotargetserver test1              data_0        1       4096       19.88       12.13     37.91
2/27/2021 5:28:41 PM ugdemotargetserver test1              XTP       65537          0                                  
2/27/2021 5:28:41 PM ugdemotargetserver test1              log           2       1024        1.74        6.26     78.25
2/27/2021 5:28:40 PM ugdemotargetserver WideWorl

dImporters data_0        1      88064      672.25       15.75      2.29
2/27/2021 5:28:40 PM ugdemotargetserver WideWorldImporters XTP       65537          0                                  
2/27/2021 5:28:40 PM ugdemotargetserver WideWorldImporters log           2     152576        6.38     1185.63     99.47
2/27/2021 5:28:40 PM ugdemotargetserver testRunBookDB      data_0        1       4096       20.06       11.94     37.31
2/27/2021 5:28:40 PM ugdemotargetserver testRunBookDB      XTP       65537          0                                  
2/27/2021 5:28:40 PM ugdemotargetserver testRunBookDB      log           2       3072        6.75       17.25     71.88
2/27/2021 5:28:38 PM ugdemotargetserver adventureworks     data_0        1       4096       26.88        5.13     16.03
2/27/2021 5:28:38 

PM ugdemotargetserver adventureworks     XTP       65537          0                                  
2/27/2021 5:28:38 PM ugdemotargetserver adventureworks     log           2       1024        1.51        6.49     81.13
2/27/2021 5:28:38 PM ugdemotargetserver master             data_0        1       4096       21.13       10.88     34.00
2/27/2021 5:28:38 PM ugdemotargetserver master             log           2       1024        0.48        7.52     94.00
2/27/2021 5:28:37 PM ugdemojobserver    dbawarehouse       data_0        1       4096       20.75       11.25

     35.16
2/27/2021 5:28:37 PM ugdemojobserver    dbawarehouse       XTP       65537          0                                  
2/27/2021 5:28:37 PM ugdemojobserver    dbawarehouse       log           2       1024        1.61        6.39     79.88
2/27/2021 5:28:36 PM ugdemojobserver    jobdatabase        log  

         2       1024        2.49        5.51     68.88
2/27/2021 5:28:36 PM ugdemojobserver    jobdatabase        data_0        1       4096       23.19        8.81     27.53
2/27/2021 5:28:36 PM ugdemojobserver    jobdatabase        XTP       65537          0                                  




### Demo  Windows Task Scheduler

Run task: '3\_WindowsTask\_CollectAzureDatabaseSize' from Windows Task Scheduler

  

If we want to do  the same and we do not have any on-premises SQL Server; we can schedule the same using any windows host. Similary we can do from an IaaS VM.

Following xml will create the task. You will need change few values.

  

<span style="font-size: 14px;">&lt;?xml version="1.0" encoding="UTF-16"?&gt;</span>

<span style="font-size: 14px;">&lt;Task version="1.4" xmlns="http://schemas.microsoft.com/windows/2004/02/mit/task"&gt;</span>

 <span style="font-size: 14px;">&nbsp; &lt;RegistrationInfo&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;Date&gt;2019-03-16T11:12:04.0903061&lt;/Date&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;Author&gt;MyLaptopName\taiob&lt;/Author&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;URI&gt;\3_WindowsTask_CollectAzureDatabaseSize&lt;/URI&gt;</span>

 <span style="font-size: 14px;">&nbsp; &lt;/RegistrationInfo&gt;</span>

 <span style="font-size: 14px;">&nbsp; &lt;Triggers&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;CalendarTrigger&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &nbsp; &lt;StartBoundary&gt;2019-03-16T10:00:00-05:00&lt;/StartBoundary&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &nbsp; &lt;Enabled&gt;true&lt;/Enabled&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &nbsp; &lt;ScheduleByDay&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &nbsp; &nbsp; &lt;DaysInterval&gt;1&lt;/DaysInterval&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &nbsp; &lt;/ScheduleByDay&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;/CalendarTrigger&gt;</span>

 <span style="font-size: 14px;">&nbsp; &lt;/Triggers&gt;</span>

 <span style="font-size: 14px;">&nbsp; &lt;Principals&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;Principal id="Author"&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &nbsp; &lt;UserId&gt;S-1-5-21-1134363470-949122783-3418734209-1001&lt;/UserId&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &nbsp; &lt;LogonType&gt;Password&lt;/LogonType&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &nbsp; &lt;RunLevel&gt;HighestAvailable&lt;/RunLevel&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;/Principal&gt;</span>

 <span style="font-size: 14px;">&nbsp; &lt;/Principals&gt;</span>

 <span style="font-size: 14px;">&nbsp; &lt;Settings&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;MultipleInstancesPolicy&gt;IgnoreNew&lt;/MultipleInstancesPolicy&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;DisallowStartIfOnBatteries&gt;true&lt;/DisallowStartIfOnBatteries&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;StopIfGoingOnBatteries&gt;false&lt;/StopIfGoingOnBatteries&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;AllowHardTerminate&gt;true&lt;/AllowHardTerminate&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;StartWhenAvailable&gt;false&lt;/StartWhenAvailable&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;RunOnlyIfNetworkAvailable&gt;false&lt;/RunOnlyIfNetworkAvailable&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;IdleSettings&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &nbsp; &lt;StopOnIdleEnd&gt;true&lt;/StopOnIdleEnd&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &nbsp; &lt;RestartOnIdle&gt;false&lt;/RestartOnIdle&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;/IdleSettings&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;AllowStartOnDemand&gt;true&lt;/AllowStartOnDemand&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;Enabled&gt;true&lt;/Enabled&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;Hidden&gt;false&lt;/Hidden&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;RunOnlyIfIdle&gt;false&lt;/RunOnlyIfIdle&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;DisallowStartOnRemoteAppSession&gt;false&lt;/DisallowStartOnRemoteAppSession&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;UseUnifiedSchedulingEngine&gt;true&lt;/UseUnifiedSchedulingEngine&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;WakeToRun&gt;false&lt;/WakeToRun&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;ExecutionTimeLimit&gt;PT2H&lt;/ExecutionTimeLimit&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;Priority&gt;7&lt;/Priority&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;RestartOnFailure&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &nbsp; &lt;Interval&gt;PT1M&lt;/Interval&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &nbsp; &lt;Count&gt;3&lt;/Count&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;/RestartOnFailure&gt;</span>

 <span style="font-size: 14px;">&nbsp; &lt;/Settings&gt;</span>

 <span style="font-size: 14px;">&nbsp; &lt;Actions Context="Author"&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;Exec&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &nbsp; &lt;Command&gt;"C:\Program Files\PowerShell\7\pwsh.exe"&lt;/Command&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &nbsp; &lt;Arguments&gt;-File "C:\Presentation\Azure SQL Database - Where is my&nbsp; SQL Agent\2_CollectDatabaseSize.ps1"&lt;/Arguments&gt;</span>

 <span style="font-size: 14px;">&nbsp; &nbsp; &lt;/Exec&gt;</span>

 <span style="font-size: 14px;">&nbsp; &lt;/Actions&gt;</span>

<span style="font-size: 14px;">&lt;/Task&gt;</span>

### Demo  Azure Automation

- Create an Automation account
- Create an empty runbook type PowerShellWorkflow

In [9]:
$automationAccountName = "ugdemo2"
$credentialName = "sqlservercredentials"
$password = Get-Content "C:\password.txt"
$passwordSecure =ConvertTo-SecureString $password -AsPlainText -Force
$credential = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList $adminlogin, $passwordSecure

#Creating an automation account
New-AzAutomationAccount `
    -ResourceGroupName $resourceGroupName `
    -Name $automationAccountName `
    -Location $primaryLocation

#Creating credential
New-AzAutomationCredential `
    -AutomationAccountName $automationAccountName `
    -Name $credentialName `
    -Value $credential `
    -ResourceGroupName $resourceGroupName

#Creating a blank runbook
New-AzAutomationRunbook `
    -ResourceGroupName $resourceGroupName `
    -AutomationAccountName $automationAccountName `
    -Name 'Update-SQLIndexRunbook' `
    -Type 'PowerShellWorkflow' `
    -LogProgress $true `
    -LogVerbose $true



SubscriptionId        : 18d92f52-ac34-4379-ab8b-5a5106f1c54e
ResourceGroupName     : sqlagentdemo
AutomationAccountName : ugdemo2
Location              : East US
State                 : Ok
Plan                  : Basic
CreationTime          : 2/27/2021 9:14:13 AM -05:00
LastModifiedTime      : 2/27/2021 9:14:13 AM -05:00
LastModifiedBy        : 
Tags                  : {}



UserName              : taiob
ResourceGroupName     : sqlagentdemo
AutomationAccountName : ugdemo2
Name                  : sqlservercredentials
CreationTime          : 2/27/2021 9:14:15 AM -05:00
LastModifiedTime      : 2/27/2021 9:14:15 AM -05:00
Description           : 



Location              : East US
Tags                  : {}
JobCount              : 0
RunbookType           : PowerShellWorkflow
Parameters            : {}
LogVerbose            : True
LogProgress           : True
LastModifiedBy        : 
State                 : New
ResourceGroupName     : sqlagentdemo
AutomationAccountName : ugdemo2
Name                  : Update-SQLIndexRunbook
CreationTime          : 2/27/2021 9:14:18 AM -05:00
LastModifiedTime      : 2/27/2021 9:14:18 AM -05:00
Description           : 



Go to portal and demo runbook

Replace the generic code with PowerShell code from below cell

**Demo Runbook** using 3\_TestRunBook.sql

In [ ]:
<#
.SYNOPSIS 
    Indexes tables in a database if they have a high fragmentation

.DESCRIPTION
    This runbook indexes all of the tables in a given database if the fragmentation is
    above a certain percentage. 
    It highlights how to break up calls into smaller chunks, 
    in this case each table in a database, and use checkpoints. 
    This allows the runbook job to resume for the next chunk of work even if the 
    fairshare feature of Azure Automation puts the job back into the queue every 30 minutes

.PARAMETER SqlServer
    Name of the SqlServer

.PARAMETER Database
    Name of the database
    
.PARAMETER SQLCredentialName
    Name of the Automation PowerShell credential setting from the Automation asset store. 
    This setting stores the username and password for the SQL Azure server

.PARAMETER FragPercentage
    Optional parameter for specifying over what percentage fragmentation to index database
    Default is 20 percent
 
 .PARAMETER RebuildOffline
    Optional parameter to rebuild indexes offline if online fails 
    Default is false
    
 .PARAMETER Table
    Optional parameter for specifying a specific table to index
    Default is all tables
    
.PARAMETER SqlServerPort
    Optional parameter for specifying the SQL port 
    Default is 1433
    
.EXAMPLE
    Update-SQLIndexRunbook -SqlServer "server.database.windows.net" -Database "Finance" -SQLCredentialName "FinanceCredentials"

.EXAMPLE
    Update-SQLIndexRunbook -SqlServer "server.database.windows.net" -Database "Finance" -SQLCredentialName "FinanceCredentials" -FragPercentage 30

.EXAMPLE
    Update-SQLIndexRunbook -SqlServer "server.database.windows.net" -Database "Finance" -SQLCredentialName "FinanceCredentials" -Table "Customers" -RebuildOffline $True

.NOTES
    AUTHOR: System Center Automation Team
    LASTEDIT: Oct 8th, 2014 
    Modified By: Taiob Ali
    Last Modified: March 2019
#>
workflow Update-SQLIndexRunbook
{
    param(
        [parameter(Mandatory=$True)]
        [string] $SqlServer = "ugdemotargetserver.database.windows.net",
    
        [parameter(Mandatory=$True)]
        [string] $Database = "testrunbookdb",
    
        [parameter(Mandatory=$True)]
        [string] $SQLCredentialName = "sqlservercredentials",
            
        [parameter(Mandatory=$False)]
        [int] $FragPercentage = 20,

        [parameter(Mandatory=$False)]
        [int] $SqlServerPort = 1433,
        
        [parameter(Mandatory=$False)]
        [boolean] $RebuildOffline = $False,

        [parameter(Mandatory=$False)]
        [string] $Table = "testRebuild"
                  
    )

    # Get the stored username and password from the Automation credential
    $SqlCredential = Get-AutomationPSCredential -Name $SQLCredentialName
    if ($SqlCredential -eq $null)
    {
        throw "Could not retrieve '$SQLCredentialName' credential asset. Check that you created this first in the Automation service."
    }
    
    $SqlUsername = $SqlCredential.UserName 
    $SqlPass = $SqlCredential.GetNetworkCredential().Password
    
    $TableNames = Inlinescript {
      
        # Define the connection to the SQL Database
        $Conn = New-Object System.Data.SqlClient.SqlConnection("Server=tcp:$using:SqlServer,$using:SqlServerPort;Database=$using:Database;User ID=$using:SqlUsername;Password=$using:SqlPass;Trusted_Connection=False;Encrypt=True;Connection Timeout=30;")
         
        # Open the SQL connection
        $Conn.Open()
        
        # SQL command to find tables and their average fragmentation
        $SQLCommandString = @"
        SELECT a.object_id, avg_fragmentation_in_percent
        FROM sys.dm_db_index_physical_stats (
               DB_ID(N'$Database')
             , OBJECT_ID(0)
             , NULL
             , NULL
             , NULL) AS a
        JOIN sys.indexes AS b 
        ON a.object_id = b.object_id AND a.index_id = b.index_id;
"@
        # Return the tables with their corresponding average fragmentation
        $Cmd=new-object system.Data.SqlClient.SqlCommand($SQLCommandString, $Conn)
        $Cmd.CommandTimeout=120
        
        # Execute the SQL command
        $FragmentedTable=New-Object system.Data.DataSet
        $Da=New-Object system.Data.SqlClient.SqlDataAdapter($Cmd)
        [void]$Da.fill($FragmentedTable)

 
        # Get the list of tables with their object ids
        $SQLCommandString = @"
        SELECT  t.name AS TableName, t.OBJECT_ID FROM sys.tables t
"@

        $Cmd=new-object system.Data.SqlClient.SqlCommand($SQLCommandString, $Conn)
        $Cmd.CommandTimeout=120

        # Execute the SQL command
        $TableSchema =New-Object system.Data.DataSet
        $Da=New-Object system.Data.SqlClient.SqlDataAdapter($Cmd)
        [void]$Da.fill($TableSchema)


        # Return the table names that have high fragmentation
        ForEach ($FragTable in $FragmentedTable.Tables[0])
        {
            Write-Verbose ("Table Object ID:" + $FragTable.Item("object_id"))
            Write-Verbose ("Fragmentation:" + $FragTable.Item("avg_fragmentation_in_percent"))
            
            If ($FragTable.avg_fragmentation_in_percent -ge $Using:FragPercentage)
            {
                # Table is fragmented. Return this table for indexing by finding its name
                ForEach($Id in $TableSchema.Tables[0])
                {
                    if ($Id.OBJECT_ID -eq $FragTable.object_id.ToString())
                     {
                        # Found the table name for this table object id. Return it
                        Write-Verbose ("Found a table to index! : " +  $Id.Item("TableName"))
                        $Id.TableName
                    }
                }
            }
        }

        $Conn.Close()
    }

    # If a specific table was specified, then find this table if it needs to indexed, otherwise
    # set the TableNames to $null since we shouldn't process any other tables.
    If ($Table)
    {
        Write-Verbose ("Single Table specified: $Table")
        If ($TableNames -contains $Table)
        {
            $TableNames = $Table
        }
        Else
        {
            # Remove other tables since only a specific table was specified.
            Write-Verbose ("Table not found: $Table")
            $TableNames = $Null
        }
    }

    # Interate through tables with high fragmentation and rebuild indexes
    ForEach ($TableName in $TableNames)
    {
      Write-Verbose "Creating checkpoint"
      Checkpoint-Workflow
      Write-Verbose "Indexing Table $TableName..."
      
      InlineScript {
          
        $SQLCommandString = @"
        EXEC('ALTER INDEX ALL ON $Using:TableName REBUILD with (ONLINE=ON)')
"@

        # Define the connection to the SQL Database
        $Conn = New-Object System.Data.SqlClient.SqlConnection("Server=tcp:$using:SqlServer,$using:SqlServerPort;Database=$using:Database;User ID=$using:SqlUsername;Password=$using:SqlPass;Trusted_Connection=False;Encrypt=True;Connection Timeout=30;")
        
        # Open the SQL connection
        $Conn.Open()

        # Define the SQL command to run. In this case we are getting the number of rows in the table
        $Cmd=new-object system.Data.SqlClient.SqlCommand($SQLCommandString, $Conn)
        # Set the Timeout to be less than 30 minutes since the job will get queued if > 30
        # Setting to 25 minutes to be safe.
        $Cmd.CommandTimeout=1500

        # Execute the SQL command
        Try 
        {
            $Ds=New-Object system.Data.DataSet
            $Da=New-Object system.Data.SqlClient.SqlDataAdapter($Cmd)
            [void]$Da.fill($Ds)
        }
        Catch
        {
            if (($_.Exception -match "offline") -and ($Using:RebuildOffline) )
            {
                Write-Verbose ("Building table $Using:TableName offline")
                $SQLCommandString = @"
                EXEC('ALTER INDEX ALL ON $Using:TableName REBUILD')
"@              

                # Define the SQL command to run. 
                $Cmd=new-object system.Data.SqlClient.SqlCommand($SQLCommandString, $Conn)
                # Set the Timeout to be less than 30 minutes since the job will get queued if > 30
                # Setting to 25 minutes to be safe.
                $Cmd.CommandTimeout=1500

                # Execute the SQL command
                $Ds=New-Object system.Data.DataSet
                $Da=New-Object system.Data.SqlClient.SqlDataAdapter($Cmd)
                [void]$Da.fill($Ds)
            }
            Else
            {
                # Will catch the exception here so other tables can be processed.
                Write-Error "Table $Using:TableName could not be indexed. Investigate indexing each index instead of the complete table $_"
             }
        }
        # Close the SQL connection
        $Conn.Close()
      }  
    }

    Write-Verbose "Finished Indexing"
}




### Demo Elastic Job Agent

<span style="font-size: 14px;">This script will create an</span> elastic job agent      

<span style="font-size: 14px;">Credit:</span>

- <span style="font-size: 14px;"><a href="https://docs.microsoft.com/en-us/azure/sql-database/elastic-jobs-powershell#create-the-elastic-job-agent" data-href="https://docs.microsoft.com/en-us/azure/sql-database/elastic-jobs-powershell#create-the-elastic-job-agent" title="https://docs.microsoft.com/en-us/azure/sql-database/elastic-jobs-powershell#create-the-elastic-job-agent">https://docs.microsoft.com/en-us/azure/sql-database/elastic-jobs-powershell#create-the-elastic-job-agent</a></span>
- <span style="font-size: 14px;"><a href="https://docs.microsoft.com/en-us/powershell/module/az.sql/new-azsqlelasticjobagent?view=azps-5.4.0" data-href="https://docs.microsoft.com/en-us/powershell/module/az.sql/new-azsqlelasticjobagent?view=azps-5.4.0" title="https://docs.microsoft.com/en-us/powershell/module/az.sql/new-azsqlelasticjobagent?view=azps-5.4.0">https://docs.microsoft.com/en-us/powershell/module/az.sql/new-azsqlelasticjobagent?view=azps-5.4.0</a></span> Use 4\_ElasticJobAgent.sql to demo Elastic Job Agent

In [10]:
New-AzSqlElasticJobAgent `
    -ResourceGroupName 'sqlagentdemo' `
    -ServerName 'ugdemojobserver' `
    -DatabaseName 'jobdatabase' `
    -Name 'agentdemo'

ResourceGroupName ServerName      DatabaseName AgentName State Tags
----------------- ----------      ------------ --------- ----- ----
sqlagentdemo      ugdemojobserver jobdatabase  agentdemo Ready     




### Demo Azure Function

Create a function app in portal with 'Managed Servie Identity'

<mark>Prerequisite:</mark>

Create a function app in portal with 'Managed Servie Identity'  

Name: SC2021

Run the code in Azure SQL Server 'ugdemotargetserver.database.windows.net'  

<span style="font-size: 14px;">Database Name:testRunBookDB&nbsp;</span> 

`CREATE USER SC2021 FROM EXTERNAL PROVIDER`

`GO`

`ALTER ROLE db_owner ADD MEMBER SC2021`

`GO`

<span style="font-size: 14px;">Replace the built in code for Azure function</span>

<span style="font-size: 14px;">Please refer to my blog post about setting up a function app</span>

<span style="font-size: 14px;"><a href="http://sqlworldwide.com/how-to-use-managed-identity-with-azure-function-app/" data-href="http://sqlworldwide.com/how-to-use-managed-identity-with-azure-function-app/" title="http://sqlworldwide.com/how-to-use-managed-identity-with-azure-function-app/">http://sqlworldwide.com/how-to-use-managed-identity-with-azure-function-app/</a></span>

**Demo Azure Function** using 3\_TestRunBook.sql

In [ ]:
# Input bindings are passed in via param block.
param($Timer)

# Get the current universal time in the default string format
$currentUTCtime = (Get-Date).ToUniversalTime()

# The 'IsPastDue' porperty is 'true' when the current function invocation is later than scheduled.
if ($Timer.IsPastDue) {
    Write-Host "PowerShell timer is running late!"
}
<#
 This function app is using 'Managed Service Identity' to connect to the Azure SQL Database
 In the interest of time I will not show the set up
 Please see details from my blog post:
 http://sqlworldwide.com/how-to-use-managed-identity-with-azure-function-app/
 
 Used help from following resources in setting up 'Managed Service Identity'
 https://docs.microsoft.com/en-us/azure/app-service/app-service-web-tutorial-connect-msi
 https://www.azurecorner.com/using-managed-service-identity-in-azure-functions-to-access-azure-sql-database/
 https://docs.microsoft.com/en-us/azure/app-service/overview-managed-identity?tabs=powershell
#>

$resourceURI = "https://database.windows.net/"
$tokenAuthURI = $env:MSI_ENDPOINT + "?resource=$resourceURI&api-version=2017-09-01"
$tokenResponse = Invoke-RestMethod -Method Get -Headers @{"Secret" = "$env:MSI_SECRET" } -Uri $tokenAuthURI
$accessToken = $tokenResponse.access_token

$SqlConnection = New-Object System.Data.SqlClient.SqlConnection
$SqlConnection.ConnectionString = "Data Source =ugdemotargetserver.database.windows.net ; Initial Catalog = testRunBookDB"
$SqlConnection.AccessToken = $AccessToken
$SqlConnection.Open()

$SqlCmd = New-Object System.Data.SqlClient.SqlCommand
$SqlCmd.CommandText = "ALTER INDEX ALL ON testRebuild REBUILD;"
$SqlCmd.Connection = $SqlConnection
$SqlAdapter = New-Object System.Data.SqlClient.SqlDataAdapter
$SqlAdapter.SelectCommand = $SqlCmd
$DataSet = New-Object System.Data.DataSet
$SqlAdapter.Fill($DataSet) 

<span style="font-size: 14px;">Clean up by removing resource group name</span>

In [ ]:
Remove-AzResourceGroup -ResourceGroupName $resourceGroupName